In [1]:
import sys
sys.path.append('../')
sys.path.append('../trimeshpy/')
sys.path.append('../surfaceflow/')

import nibabel as nb
import numpy as np

import trimeshpy
from trimeshpy.trimeshflow_vtk import TriMeshFlow_Vtk
from trimeshpy.vtk_util import lines_to_vtk_polydata, save_polydata

import time

In [3]:
surface_file = '/Users/kristianeschenburg/Desktop/Research/Data/Surfaces/285345.L.white.32k_fs_LR.acpc_dc.surf.gii'
surface = nb.load(surface_file)

triangles = surface.darrays[1].data
vertices = surface.darrays[0].data

In [4]:
#mesh = TriMesh_Vtk(triangles=triangles,vertices=vertices)
tri_mesh_flow = TriMeshFlow_Vtk(triangles, vertices)

In [5]:
# Test parameters
nb_step = 10
diffusion_step = 5
saved_flow = trimeshpy.data.output_test_flow
saved_fib = trimeshpy.data.output_test_fib


In [6]:
# Test functions
start = time.time()
#points = tri_mesh_flow.laplacian_smooth(nb_step, diffusion_step, l2_dist_weighted=False, area_weighted=False, backward_step=False, flow_file=saved_flow)
#points = tri_mesh_flow.curvature_normal_smooth(nb_step, diffusion_step, area_weighted=True, backward_step=True, flow_file=saved_flow)
#points = tri_mesh_flow.positive_curvature_normal_smooth(nb_step, diffusion_step, area_weighted=True, backward_step=True, flow_file=saved_flow)
#points = tri_mesh_flow.mass_stiffness_smooth(nb_step, diffusion_step, flow_file=saved_flow)
points = tri_mesh_flow.positive_mass_stiffness_smooth(nb_step, diffusion_step, flow_file='/Users/kristianeschenburg/Desktop/Test.dat')
#points = tri_mesh_flow.volume_mass_stiffness_smooth(nb_step, diffusion_step, flow_file=saved_flow)
stop = time.time()
print (stop - start)

 step 10 on 10 done 
10.3335542678833


In [7]:
lines = np.memmap('/Users/kristianeschenburg/Desktop/Test.dat', dtype=np.float64, 
                  mode='r', shape=(nb_step, vertices.shape[0], vertices.shape[1]))

tri_mesh_flow.set_vertices_flow(np.array(lines))
tri_mesh_flow.display(display_name="Trimeshpy: Flow resulting surface")
tri_mesh_flow.display_vertices_flow(display_name="Trimeshpy: Flow visualization")

/Users/kristianeschenburg/anaconda3/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [8]:
np.asarray(lines)
np.save('/Users/kristianeschenburg/Desktop/FlowLines',arr=np.asarray(lines))

In [9]:
da = surface.darrays[0]

for l in np.arange(lines.shape[0]):
    
    coords = np.asarray(lines[l,:,:]).astype(np.float32)
    da.data = coords
    surface.darrays[0] = da
    
    fp = '/Users/kristianeschenburg/Desktop/Flow.L.{:}.surf.gii'.format(l+1)
    nb.gifti.write(filename=fp,image=surface)

/Users/kristianeschenburg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: giftiio.write function deprecated. Use nibabel.load() instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  # Remove the CWD from sys.path while we load stuff.


In [154]:
meta_dict = {'AnatomicalStructurePrimary': 'CortexLeft',
                                    'AnatomicalStructureSecondary': 'GrayWhite',
                                     'Caret-Version': '5.65',
                                     'GeometricType': 'Anatomical',
                                     'topo_file': 'Conte69.L.32k_fs_LR.topo.gii'}

In [155]:
meta = nb.gifti.gifti.GiftiMetaData()

for k,v in meta_dict.items():
    nvp = nb.gifti.gifti.GiftiNVPairs(name=k,value=v)
    meta.data.append(nvp)
    

In [160]:
coordsys = nb.gifti.gifti.GiftiCoordSystem(dataspace='NIFTI_XFORM_TALAIRACH',xformspace='NIFTI_XFORM_TALAIRACH',xform=np.eye(4))

da0 = nb.gifti.gifti.GiftiDataArray(intent='NIFTI_INTENT_POINTSET',
                                    datatype='NIFTI_TYPE_FLOAT32',
                                    coordsys=coordsys,
                                    data=vertices,
                                    encoding='GZipBase64Binary',
                                    meta = meta)

da1 = nb.gifti.gifti.GiftiDataArray(intent='NIFTI_INTENT_TRIANGLE',
                                    datatype='NIFTI_TYPE_INT32',
                                    coordsys=coordsys,
                                    data=triangles,
                                    encoding='GZipBase64Binary')

S = nb.gifti.gifti.GiftiImage(darrays=[da0,da1])

In [161]:
S.to_filename('/Users/kristianeschenburg/Desktop/TestSaveFile.surf.gii')

In [1]:
from niio import write

In [2]:
write.save_surf?

Signature: write.save_surf(vertices, triangles, output, hemisphere=None)
Docstring:
Save a list of vertices and triangles to a surface file.

Parameters:
- - - - -
vertices: array, float
    3d coordinates of mesh points
triangles: array, int
    list of mesh triangles
output: string
    output file name
hemisphere: string
    'CortexLeft' or 'CortexRight'
File:      ~/Documents/Code/niio/niio/write.py
Type:      function
